In [4]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [5]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from distance_matching_lowrank import DistanceMatching as DistanceMatchingLR
from distance_matching_functional_bkp import DistanceMatching
import functions
import utils
import datetime
sys.path.append("../baselines/")
from baselines import baselines

from sklearn.metrics import r2_score, mean_squared_error
mse = mean_squared_error

try:
    from sklearn.manifold import TSNE
except Error:
    import warnings
    warnings.filterwarnings("ignore", message="numpy.dtype size changed")
    warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
    from sklearn.manifold import TSNE


# Finished experiment with splitting by date - baselines terrible, personalized ok!
# TODO: Improve data generation process to only have price movement.
resplit_data = False
remake_delta_Z = resplit_data
remake_dz_test_train = resplit_data

refit_lr = resplit_data
refit_mixture = resplit_data
refit_vc = resplit_data

should_plot = True
if should_plot:
    import matplotlib.pyplot as plt
    %matplotlib inline

In [6]:
npz_ar = np.load("train_test_split.npz")
X_train = npz_ar["arr_0"]
X_test = npz_ar["arr_1"]
Y_train = npz_ar["arr_2"]
Y_test = npz_ar["arr_3"]
Z_train = npz_ar["arr_4"]
Z_test = npz_ar["arr_5"]
train_idx = npz_ar["arr_6"]
test_idx = npz_ar["arr_7"]

In [7]:
beta_hat_dmr = np.load("beta_hat_dmr_lr.npy")

In [8]:
print(set(np.argmax(beta_hat_dmr, axis=1)))

{383, 513, 898, 520, 524, 657, 147, 533, 918, 791, 23, 281, 796, 286, 927, 417, 418, 555, 45, 305, 689, 819, 179, 694, 184, 825, 963, 453, 839, 329, 203, 77, 207, 737, 611, 227, 101, 485, 359, 1000, 996, 995, 249, 1019, 635, 125, 893, 510}


In [10]:
securities = ['VOO', 'SPY', 'YINN', 'XIV', 'IEO', 'CHIX', 'YANG', 'RDS-A', 'OA', 'BRK-B', 'TSLA', 'JPM', 'AAPL', 'HSBC', 'GOOGL', 'NVDA', 'E', 'BP', 'WMT', 'AMZN', 'LMT', 'MU', 'SNP', 'GM', 'FB', 'GOOG']
print(len(securities))

26


In [16]:
import datetime
fo = lambda x: str(datetime.datetime.fromordinal(x))[:-9]
    
def plot_models_over_time(security):
    mu_idx = np.where(Z_train[:, 0] == security)[0]
    #fig = plt.figure(figsize=(6,6))
    fig = plt.figure()
    ax = plt.gca()
    handle = ax.matshow(beta_hat_dmr[mu_idx, :], cmap="RdBu")
    y_ticks = ax.get_yticks()
    
    times = [Z_train[int(mu_idx[int(x)]), -1] for x in y_ticks[1:-1]]
    ticks = ['']
    ticks.extend(list(map(fo, list(map(int, times)))))
    ticks.append('')
    for i in range(1, len(ticks)-1):
        if len(ticks[i-1]) > 4:
            #try:
            if int(ticks[i][:4])*100 + int(ticks[i][5:7]) < int(ticks[i-1][:4])*100 + int(ticks[i-1][5:7]) + 100:
                #if int(ticks[i][5:7]) < :
                ticks[i] = ''
            #except ValueError:
            #    continue
    ax.set_yticklabels(ticks)
    cbar = fig.colorbar(handle)
    plt.tight_layout()
    plt.savefig("results/figures/models_over_time/models_over_time_{}.pdf".format(security), dpi=300)
    plt.close()

for security in securities:
    plot_models_over_time(security)

In [17]:
plt.rcParams['figure.max_open_warning'] = 30
def feat_name(x):
    if x >= 52:
        return ("News", x - 52)
    security = securities[int(x / 2)]
    if x % 2 == 0:
        return (security, "Price")
    else:
        return (security, "Vol.")

import pandas
from collections import Counter    

for security in securities:
    #print(security)
    my_idx = Z_train[:, 0] == security
    my_beta = beta_hat_dmr[my_idx]
    my_features = [np.argmax(x) for x in my_beta]
    my_feature_names = [feat_name(x % 102) for x in list(my_features)]
    #print(my_feature_names)
    letter_counts = Counter(my_feature_names)
    #fig = plt.figure(figsize=(8,8))
    df = pandas.DataFrame.from_dict(letter_counts, orient='index')
    df.plot(kind='bar', legend=False)
    plt.xlabel("Most Predictive Feature", fontsize=18)
    plt.ylabel("Number of Samples", fontsize=18)
    plt.tight_layout()
    plt.savefig("results/features_hist_{}.pdf".format(security), dpi=300)
    plt.close()
    #plt.title(security)
    
    
    #my_features = reversed(np.argsort(np.sum(np.abs(my_beta), axis=0)))
    #print(security, my_feature_names)

In [14]:
import umap.umap_ as umap
from sklearn.manifold import TSNE

In [15]:
t = TSNE(n_iter=750, verbose=1)
#u = umap.UMAP()
beta_hat_small = t.fit_transform(beta_hat_dmr)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 12735 samples in 0.536s...
[t-SNE] Computed neighbors for 12735 samples in 133.543s...
[t-SNE] Computed conditional probabilities for sample 1000 / 12735
[t-SNE] Computed conditional probabilities for sample 2000 / 12735
[t-SNE] Computed conditional probabilities for sample 3000 / 12735
[t-SNE] Computed conditional probabilities for sample 4000 / 12735
[t-SNE] Computed conditional probabilities for sample 5000 / 12735
[t-SNE] Computed conditional probabilities for sample 6000 / 12735
[t-SNE] Computed conditional probabilities for sample 7000 / 12735
[t-SNE] Computed conditional probabilities for sample 8000 / 12735
[t-SNE] Computed conditional probabilities for sample 9000 / 12735
[t-SNE] Computed conditional probabilities for sample 10000 / 12735
[t-SNE] Computed conditional probabilities for sample 11000 / 12735
[t-SNE] Computed conditional probabilities for sample 12000 / 12735
[t-SNE] Computed conditional probabilities for s

PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.